## Configuring

### Configuration Files

Like the Jupyter Notebook server, JupyterHub, and other Jupyter interactive
computing tools, `jupyter-lsp` can be configured via [Python or JSON
files][notebook-config] in _well-known locations_. You can find out where to put
them on your system with:

[notebook-config]: https://jupyter-notebook.readthedocs.io/en/stable/config.html

```bash
jupyter --paths
```

They will be merged from bottom to top, and the directory where you launch your
`notebook` server wins, making it easy to check in to version control.

### Configuration Options

#### language_servers

`jupyter-lsp` does not come with any Language Servers! However, we will try to
use [known language servers](./Language%20Servers.ipynb) if they _are_ installed
and we know about them: you can disable this behavior by configuring
[autodetect](#autodetect).

If you don't see an implementation for the language server you need, continue
reading!

> Please consider [contributing your language server spec](./Contributing.ipynb)
> to `jupyter-lsp`!

The absolute minimum language server spec requires:

- `argv`, a list of shell tokens to launch the server in `stdio` mode (as
  opposed to `tcp`),
- the `languages` which the server will respond to, and
- the schema `version` of the spec (currently only `1`)

```python
# ./jupyter_notebook_config.json                 ---------- unique! -----------
#                                               |                              |
# or e.g.                                       V                              V
# $PREFIX/etc/jupyter/jupyter_notebook_config.d/a-language-server-implementation.json
{
  "LanguageServerManager": {
    "language_servers": {
      "a-language-server-implementation": {
        "version": 1,
        "argv": ["/absolute/path/to/a-language-server", "--stdio"],
        "languages": ["a-language"]
      }
    }
  }
}
```

A number of other options we hope to use to enrich the user experience are
available in the [schema][].

[schema]:
  https://github.com/krassowski/jupyterlab-lsp/blob/master/py_src/jupyter_lsp/schema/schema.json

More complex configurations that can't be hard-coded may benefit from the python
approach:

```python
# jupyter_notebook_config.py
import shutil

# c is a magic, lazy variable
c.LanguageServerManager.language_servers = {
    "a-language-server-implementation": {
        # if installed as a binary
        "argv": [shutil.which("a-language-server")],
        "languages": ["a-language"],
        "version": 1
    },
    "another-language-implementation": {
        # if run like a script
        "argv": [shutil.which("another-language-interpreter"), "another-language-server"],
        "languages": ["another-language"],
        "version": 1
    }
}
```

#### nodejs

> default: `None`

An absolute path to your `nodejs` executable. If `None`, `nodejs` will be
detected in a number of well-known places.

#### autodetect

> default: `True`

If `True`, `jupyter-lsp` will look for all
[known language servers](#installing-language-servers). User-configured
`language_servers` of the same implementation will be preferred over
`autodetect`ed ones.

#### node_roots

> default: `[]`

Absolute paths to search for directories named `node_modules`, such as
`nodejs`-backed language servers. The order is, roughly:

- the folder where `notebook` or `lab` was launched
- the JupyterLab `staging` folder
- wherever `conda` puts global node modules
- wherever some other conventions put it

#### extra_node_roots

> default: `[]`

Additional places `jupyter-lsp` will look for `node_modules`. These will be
checked _before_ `node_roots`, and should not contain the trailing
`node_modules`.

### Python `entry_points`

`pip`-installable packages in the same environment as the Jupyter `notebook`
server can be automatically detected as providing
[language_servers](#language_servers). These are a little more involved, but
also more powerful: see more in [Contributing](Contributing.ipynb#Specs).
Servers configured this way are loaded _before_ those defined in
[configuration files](#Configuration-Files), so that a user can fine-tune their
available servers.